## This model constuct a networkx graph from data using the following nodes and edges

### Nodes:
- Station
- Bike

### Edges:
- TripFrom (from Station to bike)
- TripTo   (bike to Station)

In [ ]:
import pandas as pd
import networkx as nx
from graphgen import fix_columns, create_graph

In [ ]:
trips_filename = '../data/201508_trip_data.csv'
stations_filename = '../data/201508_station_data.csv'

trips_df = pd.read_csv(trips_filename)
stations_df = pd.read_csv(stations_filename)

# if columns have spaces in their names we need to replace them with underscore
fix_columns(trips_df)
fix_columns(stations_df)

In [ ]:
station_mapper = {
    'nodes': [
        {
            'type'      : 'Station',
            'key'       : 'station_id',
            'attributes': ['station_id', 'name', 'lat', 'long', 'landmark']
        }
    ]
}

bike_mapper = {
    'nodes': [
        {
            'type'      : 'Bike',
            'key'       : 'Bike_#',
            'attributes': ['Bike_#']
        },
    ]
}

edges_mapper = {
    'edges': [
        {
            'type'      : 'TripFrom',
            'from'      : {
                'key'     :'Start_Terminal',
                'node_key': 'station_id'
            },
            'to'        : {
                'key'     : 'Bike_#',
                'node_key': 'Bike_#'
            },
            'attributes': ['Trip_ID', 'Start_Date']
        },
        {
            'type'      : 'TripTp',
            'from'        : {
                'key'     : 'Bike_#',
                'node_key': 'Bike_#'
            },
            'to'      : {
                'key'     :'End_Terminal',
                'node_key': 'station_id'
            },
            'attributes': ['Trip_ID', 'End_Date']
        }
    ]
}

In [ ]:
display(stations_df.head())

In [ ]:
# make sure that second set of nodes does have a unique IDs
# In this example we'll keep the stations as is, and replace the Bike # with the same number appended with 555
trips_df['Bike_#'] = trips_df['Bike_#'].apply(lambda x : 5550000 + x)

In [ ]:
display(trips_df.head())

In [ ]:

# construct a bidirectional multi-edge graph object
g = nx.MultiDiGraph()

%time g = create_graph(g, graph_mapper = station_mapper, \
                 data_provider = stations_df, update=False)

%time g = create_graph(g, graph_mapper = bike_mapper, \
                 data_provider = trips_df, update=False)

%time g = create_graph(g, graph_mapper = edges_mapper, \
                 data_provider = trips_df)

In [ ]:
# print(g.get_edge_data(50, 5550288))